## Training for Task 2

In [1]:
# Load dependencies!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F

In [67]:
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, KFold

In [63]:
df_mitbih_train = pd.read_csv("../Datasets/ecg_dataset/mitbih_train.csv", header = None)
df_mitbih_test = pd.read_csv("../Datasets/ecg_dataset/mitbih_test.csv", header = None)

In [65]:
df_mitbih_test.shape, df_mitbih_train.shape

((21892, 188), (87554, 188))

In [24]:
df_mitbih_test[187].unique()

array([0., 1., 2., 3., 4.])

In [26]:
X_train = df_mitbih_train.drop(columns=[187])

In [52]:
X_train = df_mitbih_train.drop(columns=[187])
y_train = df_mitbih_train[187]
X_test = df_mitbih_test.drop(columns=[187])
y_test = df_mitbih_test[187]


In [61]:
y_train.shape

(87554,)

In [55]:
X_test.shape, X_train.shape, y_test.shape, y_test.shape

((21892, 187), (87554, 187), (21892,), (21892,))

In [56]:
myknn = KNeighborsClassifier()
myknn.fit(X_train,y_train)
y_pred = myknn.predict(X_test)

acc = accuracy_score(y_test, y_pred)

class_report = classification_report(y_test,y_pred)



In [57]:
print(class_report)

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.99     18118
         1.0       0.90      0.64      0.75       556
         2.0       0.94      0.90      0.92      1448
         3.0       0.76      0.64      0.69       162
         4.0       0.99      0.95      0.97      1608

    accuracy                           0.97     21892
   macro avg       0.91      0.82      0.86     21892
weighted avg       0.97      0.97      0.97     21892



In [58]:
acc

0.9736433400328888

In [68]:
pipeline = Pipeline([
    ('scaling', None),
    ('model', KNeighborsClassifier())
])

# parameter grid
param_grid = {
    'model__n_neighbors': [3, 5, 7,10,15],
    'model__weights': ['uniform', 'distance']
}

#cross-validation 
cv = KFold(n_splits=3, shuffle=True, random_state=42)

# GridSearch object
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, verbose=3)

# Fit the GridSearch object 
grid_search.fit(X_train, y_train)

# Evaluate the best model
best_model_knn = grid_search.best_estimator_
y_pred_knn = best_model_knn.predict(X_test)
optimized_score_knn = best_model_knn.score(X_test, y_test)
print('Accuracy: ', optimized_score_knn)
optimized_knn_class_report = classification_report(y_test,y_pred_knn,output_dict=True)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV 1/3] END model__n_neighbors=3, model__weights=uniform;, score=0.974 total time=   5.2s
[CV 2/3] END model__n_neighbors=3, model__weights=uniform;, score=0.975 total time=   5.1s
[CV 3/3] END model__n_neighbors=3, model__weights=uniform;, score=0.974 total time=   5.1s
[CV 1/3] END model__n_neighbors=3, model__weights=distance;, score=0.974 total time=   4.7s
[CV 2/3] END model__n_neighbors=3, model__weights=distance;, score=0.975 total time=   4.8s
[CV 3/3] END model__n_neighbors=3, model__weights=distance;, score=0.975 total time=   4.8s
[CV 1/3] END model__n_neighbors=5, model__weights=uniform;, score=0.972 total time=   5.2s
[CV 2/3] END model__n_neighbors=5, model__weights=uniform;, score=0.973 total time=   5.0s
[CV 3/3] END model__n_neighbors=5, model__weights=uniform;, score=0.972 total time=   5.3s
[CV 1/3] END model__n_neighbors=5, model__weights=distance;, score=0.973 total time=   4.7s
[CV 2/3] END model__n_nei

In [69]:
best_model_knn

Pipeline(steps=[('scaling', None),
                ('model',
                 KNeighborsClassifier(n_neighbors=3, weights='distance'))])

In [70]:
!pip install lazypredict

     ---------------------------------------- 1.0/1.0 MB 13.0 MB/s eta 0:00:00
     --------------------------------------- 70.9/70.9 MB 22.6 MB/s eta 0:00:00


In [71]:
from lazypredict.Supervised import LazyClassifier

In [72]:
clf = LazyClassifier(predictions=True)
models, predictions = clf.fit(X_train,X_test,y_train,y_test)

 10%|████████▌                                                                          | 3/29 [05:15<41:08, 94.94s/it]C:\Users\oconn\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\oconn\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\oconn\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\oconn\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\oconn\anaconda3\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
 48%|████████████████

In [73]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
XGBClassifier,0.98,0.86,None,0.98,92.55
LGBMClassifier,0.98,0.85,None,0.98,7.08
KNeighborsClassifier,0.97,0.82,None,0.97,6.09
ExtraTreesClassifier,0.98,0.82,None,0.97,26.70
RandomForestClassifier,0.97,0.81,None,0.97,127.45
DecisionTreeClassifier,0.95,0.80,None,0.95,44.31
BaggingClassifier,0.97,0.80,None,0.97,252.55
ExtraTreeClassifier,0.95,0.79,None,0.95,0.85
SVC,0.97,0.77,None,0.97,203.98
